#Augmented Perspective

This iPython Notebook runs the the Augmented Perspective experiment flow

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where your augmented perspective repo is checked out
# e.g. 'projects/augmented-perspective':
# if FOLDERNAME, current directory of the iPython notebook will be used
FOLDERNAME = 'CS231A/augmented-perspective'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cd $FOLDERNAME

Then, you should be able to run the following with no problem.

In [ ]:
%load_ext autoreload
%autoreload 2

import importlib
import sys

# pre-import everything to save time
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import torch

!nvidia-smi 
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.empty_cache() 
print("Using {} for computation".format(device))

In [ ]:
# try import project libraries
import augmented_perspective
import calibration
import depth_model

Run the depth model with GPU againsts assets/kitti2.png

In [ ]:
importlib.reload(depth_model)
argv = ["runner.py", "--image_path", "assets/kitti2.png", "--device", device]
depth_model.run_depth_model(argv)
original_image = io.imread("assets/kitti2.png")
depth_image_after_monodepth2 = io.imread("outputs/kitti2_monodepth2_depth.png")
depth_map_after_monodepth2 = np.load("outputs/kitti2_monodepth2_depth.npy")
print("Depth Map Size After Monodepth:{}".format(depth_map_after_monodepth2.shape))
depth_image_after_boosting = io.imread("outputs/kitti2_boosting_depth.png")
depth_map_after_boosting = np.load("outputs/kitti2_boosting_depth.npy")
print("Depth Map Size After Boosting:{}".format(depth_map_after_boosting.shape))


In [ ]:
plt.rcParams.update({'font.size': 6})
f, ax = plt.subplots(1, 3, dpi=500)
ax[0].imshow(original_image)
ax[0].title.set_text('RGB')
ax[1].imshow(depth_image_after_monodepth2)
ax[1].title.set_text('Monodepth2')
ax[2].imshow(depth_image_after_boosting)
ax[2].title.set_text('Boosting')

Run the augmented perspective code againsts assets/kitti2.png

In [ ]:
importlib.reload(augmented_perspective)
argv = ["runner.py", "--image_path", "assets/kitti2.png", "--depth_map_path", "outputs/kitti2_monodepth2_depth.npy"]
augmented_perspective.run_augmented_perspective(argv)
reprojected_image_after_monodepth2 = io.imread("output_images/kitti2_monodepth2_depth_reprojected.png")

argv = ["runner.py", "--image_path", "assets/kitti2.png", "--depth_map_path", "outputs/kitti2_boosting_depth.npy"]
augmented_perspective.run_augmented_perspective(argv)
reprojected_image_after_boosting = io.imread("output_images/kitti2_boosting_depth_reprojected.png")


In [ ]:
plt.rcParams.update({'font.size': 6})
f, ax = plt.subplots(1, 3, dpi=500)
ax[0].imshow(original_image)
ax[0].title.set_text('RGB')
ax[1].imshow(reprojected_image_after_monodepth2)
ax[1].title.set_text('Monodepth2')
ax[2].imshow(reprojected_image_after_boosting)
ax[2].title.set_text('Boosting')